<font size="6">**Stationarity**</font>

In [5]:

import numpy as np
import seaborn as sns

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
%run ../nb_config.py

In [9]:
from src.load_data import market_data
from src.mle import time_series as mle_ts
from src import utils

# Parameters

In [11]:
cfg_brent =  utils.conf['fparams']['brent_model']

In [ ]:
TAU_TARGET = int(cfg_brent['target_tau'])

# Load Data

In [ ]:
comm_df = market_data.read_mkt_data()[['brent']]

In [ ]:
comm_df.plot(title='Brent daily close price')
plt.ylabel("USD/bbl")
plt.show()

In [ ]:
target = mle_ts.get_targets(y=comm_df[['brent']], tau=TAU_TARGET)

In [ ]:
target_adf_res = mle_ts.run_adf_test(target)
target_adf_res

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 6))

ax = axs[0]
target.plot(ax=ax)
ax = axs[1]
target.plot.hist(ax=ax, bins=100, orientation='horizontal', density=True)
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(16, 6))

ax = axs[0]
comm_df[:'2019'].plot(y ='brent', ax=ax)
ax = axs[1]
target[:'2019'].plot(ax=ax)
plt.show()

In [ ]:
from statsmodels.stats.diagnostic import kstest_normal

In [ ]:
kstest_normal(target)

In [ ]:
target.kurtosis()

In [ ]:
target.skew()

# Addendum

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

In [ ]:
x_cos = pd.DataFrame(np.cos(np.arange(0, 100)), columns=['cos'])

x_cos.plot()

In [ ]:
mle_ts.run_adf_test(x_cos)

In [ ]:
from statsmodels.tsa.stattools import adfuller

adfuller(comm_df['brent'][1:], regression='nc')

In [ ]:
import statsmodels.api as sm
import numpy as np

endog = comm_df['brent'].diff(1)[1:]
exog = comm_df['brent'].shift(1)[1:]
#exog = sm.add_constant(exog)
model = sm.OLS(endog,exog)
results = model.fit()
results.summary()